In [4]:
# design then implement 
# test everything you implement
# test fast
# code it :) 
# finish in 20 min 


# build out small test for 3 graphs. 
    # test for linear 
    # very interesting if we increase beta -> 10, then it converges around the vertex. 
    # But if we decrease beta -> 0.01, then it does not converge around the vertex but rather discrete minimums like -1, -1, 1, 1, 1,1 -1. 
    # same effect for f_quadratic and nn
    # test for quadratic
    # test for nn
    
# if you restrict your boundary, then your points on the simplex can only be within a certain range
# however, if you have an infinite boundary, then your simplex can be found. (beta means nothing)
# we want it to be within like [1, -1, -1, -1, -1] but its more like [-1, 1, -1, 1]
# beta = 10 can get to like 1 num 1, but when beta = 0.01 get 5 num 1. Non-optimal because we want our large value to be maximized.
# what if this is the global optimum*******************
# if it is -> rethink research question (which is good)
# nn -> output bounds (bounds are good, without bounds everything would be the same.) -> the global minimums are at 1, 1, -1, -1, multiple points with 1's
# understand how the distribution of nn layers are. 
# understand how the 
    
    
# run experiment with random initialization 10 times, (take the minimum) add in change in beta, add in change in dimensions, 
# graph 3 different networks and plot


# more rigorous graphs for random initializations
# try different np-solvers
# try different distributions initializations.



In [1]:
import numpy as np
from scipy.optimize import fmin_l_bfgs_b

In [299]:
def softmax_beta_x(x, beta = 1):
    return np.exp(beta * x) / np.sum(np.exp(beta * x))


def f_linear(inp, *args):
    # negative parameter is when we want to convert max f(x) -> min -f(x). 
    # The x values are the same but the optimum f(x) is negated. 
    beta, dimension, negative = args[0]
    x = softmax_beta_x(inp, beta)
    np.testing.assert_almost_equal(np.sum(x), 1.0)
    #x = inp
    w, b = args[1]
    assert(w.size == dimension and x.size == dimension)
    if negative: 
        return -(w.dot(x) + b)
    else: 
        return w.dot(x) + b
            

def f_linear_discrete(*args):
    beta, dimension, negative = args[0]
    w, b = args[1]
    assert(w.size == dimension)
    results = [w.dot(x) + b for x in np.eye(dimension)]
    return results, np.argmax(results), np.max(results)


def f_quadratic(inp, *args): 
    beta, dimension, negative = args[0]
    x = softmax_beta_x(inp, beta)
    Q, w, b = args[1]
    assert(Q.shape == (dimension, dimension) 
           and x.size == dimension 
           and w.size == dimension)
    if negative: 
        return -(x.T @ Q @ x + w.dot(x) + b)
    else: 
        return x.T @ Q @ x + w.dot(x) + b

    
def f_quadratic_discrete(*args): 
    beta, dimension, negative = args[0]
    Q, w, b = args[1]
    assert(Q.shape == (dimension, dimension) 
           and w.size == dimension)
    results = [x.T @ Q @ x + w.dot(x) + b for x in np.eye(dimension)]
    return results, np.argmax(results), np.max(results)


def f_neural_network(inp, *args): 
    beta, dimension, negative = args[0]
    x = np.array([softmax_beta_x(inp, beta)])
    for arg in args[1:]: 
        w, b, activation = arg[0], arg[1], arg[2]
        x = x.dot(w) + b; 
        if activation == 'sigmoid':
            x = 1/(1 + np.exp(-x))
        elif activation == "relu": 
            x = np.maximum(x, 0)
        else: 
            continue
    if negative: 
        return -(x[0][0])
    else: 
        return x[0][0]
    

def f_neural_network_discrete(*args): 
    beta, dimension, negative = args[0]
    results = []
    for elem in np.eye(dimension): 
        x = np.array(elem).T
        for arg in args[1:]: 
            w, b, activation = arg[0], arg[1], arg[2]
            x = x.dot(w) + b; 
            if activation == 'sigmoid':
                x = 1/(1 + np.exp(-x))
            elif activation == "relu": 
                x = np.maximum(x, 0)
            else: 
                continue
        results.append(x[0])
    return results, np.argmax(results), np.max(results)


def gen_linear_f(d, distribution):
    if distribution == 'uniform':
        return [[np.random.uniform(-1, 1, (d)), np.random.uniform(-1, 1)]]
    elif distribution == 'normal':
        return [[np.random.normal(0, 1, (d)), np.random.normal(0, 1)]]
    else: 
        print("Distribution not supported")
        return -1 


def gen_quadratic_f(d, distribution): 
    if distribution == "uniform": 
        return [[np.random.uniform(-1, 1, (d, d)), np.random.uniform(-1, 1, (d)), np.random.uniform(-1, 1)]]
    elif distribution == "normal": 
        return [[np.random.normal(0, 1, (d, d)), np.random.normal(0, 1, (d)), np.random.normal(0, 1)]]
    else: 
        print("Distribution not supported")
        return -1 
    

def gen_nn_f(d, distribution):
    weights = []
    for i in range(len(d) - 1): 
        w = np.random.normal(0, 1 / (d[i+1] ** 0.5), (d[i], d[i+1]))
        b = np.random.normal(0, 1 / (d[i+1] ** 0.5), (d[i+1]))
        if i == len(d) - 2: 
            weights.append([w, b, "none"]) 
        elif distribution == "sigmoid": 
            weights.append([w, b, "sigmoid"])
        elif distribution == "relu":
            weights.append([w, b, "relu"])
        else: 
            print("Distribution not supported")
            return []
    return weights 

In [374]:
# discrete tests done
dimensions = 3
weights = gen_linear_f(dimensions, "uniform")
#weights[0][0][0] = 100

In [376]:
#weights[0][0][0] = 1000
verbose=False
z_lst = []
for i in range(1000): 
    x0 = np.random.uniform(-1, 1, (dimensions))
    beta = 0.01
    args = tuple([(beta, dimensions, True)] + weights)
    bounds = [(-1, 1) for _ in range(dimensions)]
    z = fmin_l_bfgs_b(f_linear, x0=x0, args=args, approx_grad=True,bounds=bounds, pgtol=1e-8)
    args = list(args)
    z_lst.append(z)
    args[0] = [beta, dimensions, False]
    args = tuple(args)
    a, b = np.argwhere(weights[0][0] > 0).flatten(), np.argwhere(z[0] > 0).flatten()
    if verbose: 
        print("Input: ", z[0])
        #print("Grad < eps", np.all(np.abs(z[2]['grad']) < 1e-3))
        #print("Gradient", z[2]['grad'])
        print("Num of 1:", sum(abs(1 - x) < 1e-5 for x in z[0]))
        print("Softmax simplex: ", softmax_beta_x(z[0]))
        print("Weights: ", weights[0][0])

        print("Argmax weights: ", np.argmax(weights[0][0]), " Argmax input: ", np.argmax(z[0]))
        print("Value: ", -z[1])
        print("f linear value: ", f_linear(z[0], *args))
        print("Discrete value optimum point and value", f_linear_discrete(*args)[1], f_linear_discrete(*args)[2])
        print(a == b)
        print(a)
        print(b)
        print(len(a))
        print(len(b))


In [396]:
print(weights)
print(f_linear(np.array([-1., 1., 1.]), *args))
print(f_linear(np.array([-1., -1., 1.]), *args)) 
print(f_linear(np.array([-1., -1., -1.]), *args))
print(f_linear(np.array([1., 1., 1.]), *args))

print(z_lst[0][0])
print(-z_lst[0][1])

[[array([-0.84979242,  0.56904581,  0.73410725]), -0.02424032394425324]]
0.1335302491131125
0.13077933621517815
0.1268798877815863
0.1268798877815863
[-1.  1.  1.]
0.1335302491131125


In [383]:
tasks = np.array([z_lst[i][0] for i in range(1000)])
np.isclose(tasks, tasks[0]).all()

True

In [211]:
"""
# test quadratic
dimensions = 100
x0 = np.random.uniform(-1, 1, (dimensions))
weights = gen_quadratic_f(dimensions, "uniform")
weights[0][0][0, 0] = 1000
beta = 1.0
args = tuple([(beta, dimensions, True)] + weights)
bounds = [(-1, 1) for _ in range(dimensions)]
z = fmin_l_bfgs_b(f_quadratic, x0=x0, args=args, approx_grad=True, bounds=bounds, pgtol=1e-8)
args = list(args)
args[0] = [beta, dimensions, False]
args = tuple(args)
print(z[0])
print(-z[1])
print(f_quadratic(z[0], *args))
f_quadratic_discrete(*args)[1], f_quadratic_discrete(*args)[2]
"""


'\n# test quadratic\ndimensions = 100\nx0 = np.random.uniform(-1, 1, (dimensions))\nweights = gen_quadratic_f(dimensions, "uniform")\nweights[0][0][0, 0] = 1000\nbeta = 1.0\nargs = tuple([(beta, dimensions, True)] + weights)\nbounds = [(-1, 1) for _ in range(dimensions)]\nz = fmin_l_bfgs_b(f_quadratic, x0=x0, args=args, approx_grad=True, bounds=bounds, pgtol=1e-8)\nargs = list(args)\nargs[0] = [beta, dimensions, False]\nargs = tuple(args)\nprint(z[0])\nprint(-z[1])\nprint(f_quadratic(z[0], *args))\nf_quadratic_discrete(*args)[1], f_quadratic_discrete(*args)[2]\n'

In [122]:
"""
# test nn
dimensions = 10
x0 = np.random.uniform(-1, 1, (dimensions))
weights = gen_nn_f([dimensions, dimensions, 1], "relu")
#print(weights)
#weights[0][0][0,0] = 100
#weights[1][0][0] = 100
#print(weights)
beta = 0.1
args = tuple([(beta, dimensions, True)] + weights)
bounds = [(-1, 1) for _ in range(dimensions)]
z = fmin_l_bfgs_b(f_neural_network, x0=x0, args=args, approx_grad=True, bounds=bounds, pgtol=1e-8)
args = list(args)
args[0] = [beta, dimensions, False]
args = tuple(args)
print(z[0])
print(-z[1])
print(f_neural_network(z[0], *args))
f_neural_network_discrete(*args)[1], f_neural_network_discrete(*args)[2]
"""

'\n# test nn\ndimensions = 10\nx0 = np.random.uniform(-1, 1, (dimensions))\nweights = gen_nn_f([dimensions, dimensions, 1], "relu")\n#print(weights)\n#weights[0][0][0,0] = 100\n#weights[1][0][0] = 100\n#print(weights)\nbeta = 0.1\nargs = tuple([(beta, dimensions, True)] + weights)\nbounds = [(-1, 1) for _ in range(dimensions)]\nz = fmin_l_bfgs_b(f_neural_network, x0=x0, args=args, approx_grad=True, bounds=bounds, pgtol=1e-8)\nargs = list(args)\nargs[0] = [beta, dimensions, False]\nargs = tuple(args)\nprint(z[0])\nprint(-z[1])\nprint(f_neural_network(z[0], *args))\nf_neural_network_discrete(*args)[1], f_neural_network_discrete(*args)[2]\n'

In [ ]:
function = "linear"
beta = 0.1
trials

if function == "linear": 
    w = gen_linear_f(d=dims, distribution=weight_init_args)
    function = linear_f
elif function == "quadratic": 
    w = gen_quadratic_f(d=dims, distribution=weight_init_args)
    function = quadratic_f
elif function == "neural_network": 
    w = gen_nn_f(d=layers, distribution=weight_init_args)
    function = nn_f
else: 
    print("Invalid function choice")
    return
x0 = np.random.uniform(-1, 1, (dimensions))

output = function(x, w)
discrete_argmax_index = np.argmax(function(np.eye(dims), w))
nearest_discrete_vertex = np.argmax(x, axis = 1) 

In [312]:
def f2(z, *params):
    x, y = z
    a, b, c, d, e, f, g, h, i, j, k, l, scale = params
    return (-g*np.exp(-((x-h)**2 + (y-i)**2) / scale))

In [315]:
x0 = np.array([2., 2.])     # Initial guess.
np.random.seed(555)   # Seeded to allow replication.
res = anneal(f, x0, args=params, schedule='boltzmann',
                      full_output=True, maxiter=500, lower=-10,
                          upper=10, dwell=250, disp=True)

ImportError: cannot import name 'anneal'